In [73]:
import dask
import numpy as np
import dask.array as da
from xarrayms import xds_from_ms, xds_from_table

Load data from the Measurement Set 

**UVW** is required for calculating bl_max_dist
**ANTENNA1 and ANTENNA2** is needed to calculate the unique baseline

**xds** is a list

In [74]:
xds = list(xds_from_ms("1491291289.1GC.ms", columns=["UVW", "ANTENNA1", "ANTENNA2"], 
                       group_cols=[], index_cols=[], chunks={"row": 1e9}))

Successful readonly open of default-locked table 1491291289.1GC.ms: 26 columns, 129480 rows


In [75]:
# THis is the entire data measurement set
assert len(xds) == 1
ds = xds[0]

Calculate the maximum baseline 

In [76]:
uvw = ds.UVW.data
bl_max_dist = da.sqrt(da.max(da.sum(uvw**2, axis=1)))

In [77]:
print(bl_max_dist.compute())

7697.562658632719


Find the unique baselines

In [78]:
# Make sure dtype is int32 for both Antennas
# This is important as we will create a view of int64 from the antennas
assert ds.ANTENNA1.dtype == ds.ANTENNA2.dtype == np.int32

Stack antenna1 and antenna2 data

In [79]:
# Stack Antenna 2 data on a different axis
bl = da.stack([ds.ANTENNA1.data, ds.ANTENNA2.data], axis=1)
bl.shape

(129480, 2)

Convert array to dtype int64 from int32

In [80]:
bl = bl.rechunk(-1,2).view(np.int64)
bl.shape

(129480, 1)

Get unique values (baselines)

In [81]:
ubl = da.unique(bl)

In [82]:
# Compute --> view of int32 and reshape
ubl = da.compute(ubl)[0].view(np.int32).reshape(-1,2)
ubl.shape

(120, 2)

Reread the measurement set, taking TIME, ANTENNAS, UVW, FLAG_ROW and group them by SCAN_NUMBER

In [83]:
xds = list(xds_from_ms("1491291289.1GC.ms", columns=["TIME", "ANTENNA1", "ANTENNA2", "UVW", "FLAG_ROW"], 
                       group_cols=["SCAN_NUMBER"], index_cols=[], chunks={"row": 1e9}))

Successful readonly open of default-locked table 1491291289.1GC.ms: 26 columns, 129480 rows


In [84]:
type(xds)

list

In [85]:
valid_rows = np.empty(ubl.shape[0], dtype=np.bool)

**Averaging Function** (This is what I need to complete)

So what is needed is the following variables: 
 * **bl_max_ew**  (Maximum EW distance for the valid rows from uvw)
 * **rows** (Count number of valid rows, where valid_rows == True)
 * **bl_max_dist** (Maximum Baseline distance for the valid rows) 

With the above three variables, it should be possible to calculate baseline_avg_rows (the row size of the compressed data) <br />
**bl_max_dist** is calculated from all the uvw values (not just the valid ones)

I think the formular for this calculation is the following: <br/>
 number of valid rows divide by (EW max distance divide by Max baseline)

In [147]:
def _averaging_rows(time, ant1, ant2, uvw, flagged_rows, bl_max_dist, ubl=None):
    # TODO(smasoka)
    # Understand this
    # Need to index passed argument lists,
    # as atop contracts over dimensions not present in the output.
    ant1 = ant1[0]
    ant2 = ant2[0]
    uvw = uvw[0][0]
    flagged_rows = flagged_rows[0]

#     print ant1.shape
#     print ant2.shape
#     print uvw.shape
#     print flagged_rows.shape

    # Create empty array container for number of average rows
    # with shape of the unique baselines
    baseline_avg_rows = np.empty(ubl.shape[0], dtype=np.int32)
    # print baseline_avg_rows.shape
    
    # Create an empty array container for number of valid rows in each baseline
    # with the shape of unique baselines
    bl_num_rows = np.empty(ubl.shape[0], dtype=np.int32)
    bl_nrows = np.empty(ubl.shape[0], dtype=np.int32)
    
    unflagged = flagged_rows == False

#     print "UBL"
#     print "UBL SHAPE " +str(ubl.shape)
#     print "UBL SIZE " +str(ubl.size)
#     print

    # Foreach baseline
    for bl, (a1, a2) in enumerate(ubl):
        # print "bl : %s, a1 : %s, a2 : %s" %(bl, a1, a2)

        # Find rows associated with each baseline
        # also removing flagged rows
        valid_rows = (ant1 == a1) & (ant2 == a2) & unflagged
        # depending on the unflagged, valid_rows can be reduced, smaller
        print "VALID_ROWS " +str(valid_rows)
        print "VALID_ROWS SHAPE" +str(valid_rows.shape)
        # print
        # Maximum EW distance for each baseline
        bl_max_ew = np.abs(uvw[valid_rows, 0]).sum()
        # print "MAX EW DISTANCE " +str(bl_max_ew)
        
        # Get the number of rows for this baseline
        bl_nrows[bl] = np.where(valid_rows == True)[0].size
        # bl_nrows[bl] = valid_rows.sum()
        # print "BL ROWS " +str(type(bl_nrows))
        # print "BL ROWS" +str(bl_nrows)
        
        # Figure out what the averaged number of rows will be
        # I think np.divmod is the way to do this
        # baseline_avg_rows[i] = np.divmod(valid_rows)
        # Number (count) of valid_rows for this baseline
        # baseline_num_rows.append(da.where(valid_rows == True)[0].size)
        # basically for each ubl (120) array, baseline_avg_rows (120) each
        # containing the number of lines/rows (int) : baseline_num_rows

        # bl_num_rows, bl_max_ew and bl_max_dist are all the variables needed to do the
        # calculation.
        avg_ratio = bl_max_ew / bl_max_dist
        baseline_avg_rows, baseline_avg_rows_rem = np.divmod(valid_rows.sum(),avg_ratio)
        bl_num_rows[bl] = baseline_avg_rows
        
        print "bl_num_rows" +str(bl_num_rows[bl])
        print "bl_nrows" +str(bl_nrows[bl])
        
        # Get the remainder. 
        
    return bl_num_rows

Array container to hold all the average rows

In [148]:
scan_baseline_avg_rows = []

For loop that uses dask atop function

In [149]:
for ds in xds:
    # calls _averaging_rows
    # output block pattern ("bl",) --> array of baselines
    # TIME data (row array)
    # ANTENNA1 data (row array)
    # ANTENNA2 data (row array)
    # UVW data (row array and all columns) ???
    # FLAG_ROW data (row array)
    # creates a new_axis for the number of baselines
    # pass the actual array of unique baselines
    # dtype of results
    avg_rows = da.core.atop(_averaging_rows, ("bl",),
                            ds.TIME.data, ("row",),
                            ds.ANTENNA1.data, ("row",),
                            ds.ANTENNA2.data, ("row",),
                            ds.UVW.data, ("row", "(u,v,w)"),
                            ds.FLAG_ROW.data, ("row",),
                            bl_max_dist, (),
                            # Must tell dask about the number of baselines
                            new_axes={"bl": ubl.shape[0]},
                            # Pass through ubl to all instances of
                            ubl=ubl,
                            dtype=np.int32)

    scan_baseline_avg_rows.append(avg_rows)

Dask compute with argument "scan_baseline_avg_rows"

In [150]:
results = dask.compute(scan_baseline_avg_rows, scheduler="sync")

VALID_ROWS [ True False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows3630
bl_nrows38
VALID_ROWS [False  True False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows602
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows721
bl_nrows38
VALID_ROWS [False False  True ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows81
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows83
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows94
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows65
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows67
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows74
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
b

VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows3
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows9
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows17
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows8
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows8
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows9
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows9
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows10
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows9
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows9

bl_num_rows10
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows12
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows11
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows12
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows9
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows7
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows12
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows12
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows12
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows15
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_

VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows82
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows88
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows119
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows652
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows2545
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows121
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows208
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows11
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows11
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)

VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows5
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows5
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows5
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows6
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows7
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows314
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows4
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows2
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows1
bl_nrows37
VALID_ROWS [False False False ... False  True False]
VALID_ROWS SHAPE(4440,)
bl_num_rows3

...

bl_num_rows5
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows3
bl_nrows37
VALID_ROWS [False False False ...  True False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows11
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows6
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows6
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows6
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows6
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows5
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows6
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows6
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SH

VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows2
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows2
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows2
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows2
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows2
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows2
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows1
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows1
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows3
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows8
b

bl_num_rows9
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows9
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows8
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows8
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows9
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows8
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows8
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows4
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows4
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4560,)
bl_num_rows4
bl_nrows38
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHA

VALID_ROWS [False False False ... False False  True]
VALID_ROWS SHAPE(4560,)
bl_num_rows22
bl_nrows38
VALID_ROWS [ True False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows16498
bl_nrows37
VALID_ROWS [False  True False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows1243
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows1345
bl_nrows37
VALID_ROWS [False False  True ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows77
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows78
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows83
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows61
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows62
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,

VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows5
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows24
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows23
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows23
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows36
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows43
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows31
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows33
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num_rows20
bl_nrows37
VALID_ROWS [False False False ... False False False]
VALID_ROWS SHAPE(4440,)
bl_num

In [90]:
type(results)

tuple

In [91]:
len(results[0])

29

In [138]:
results

([array([ 824,  186,  240,  109,  126,  267,  100,  114,  219, 1204,  493,
         1226,  299,  141,  126,  178,  228, 4305,  285,  230,  280,   54,
           58,   78,  110,  121,   61,   79,   35,   33,   29,   26,   26,
           32,   29,   21,    4,    4,    4,    4,    4,    4,    4,    4,
            4,    4,    4,    4,    4,    4,    4,    4,    4,    5,   36,
           12,   12,   11,   11,   11,   12,   11,   10,   19,    6,    8,
           10,   10,   11,   11,   11,   10,   11,   13,    8,    3,    3,
            5,    7,    7,    7,    7,    7,    7,    7,    8,    5,    2,
            2,    4,   21,    6,    6,    6,    6,    6,    6,    6,    7,
            5,    2,    2,    4,   15,   57,    8,    8,    8,    8,    9,
            8,    8,    9,    6,    2,    3,    5,   37,   53,   27],
        dtype=int32),
  array([899, 201, 258, 105, 119, 221,  94, 105, 178, 936, 397, 714, 406,
         143, 124, 165, 202, 932, 289, 221, 283,  59,  64,  85, 138, 163,
          

### Next I want to compare how much compressed are these rows versus the original ones.

In [ ]:
ubl.shape

In [ ]:
valid_rows.shape

In [ ]:
type(valid_rows)

In [ ]:
valid_rows.sum()